In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../src')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("="*80)
print("EXPLORATORY DATA ANALYSIS & DATA CLEANING")
print("="*80)

''' SECTION 1: DATA LOADING AND INITIAL INSPECTION '''
print("\n1. DATA LOADING AND INITIAL INSPECTION")
print("-" * 60)

# Load the raw dataset
df = pd.read_csv('../data/raw/US_Regional_Sales_Data.csv')

print(f"📊 Dataset shape: {df.shape}")
print(f"📋 Columns ({len(df.columns)}): {list(df.columns)}")
print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display basic information
print("\n📈 DATASET OVERVIEW:")
print(df.info())

print("\n🔍 FIRST 5 ROWS:")
print(df.head())

print("\n📊 BASIC STATISTICS:")
print(df.describe())

In [ ]:
''' SECTION 2: DATA QUALITY ASSESSMENT '''
print("\n" + "="*80)
print("2. COMPREHENSIVE DATA QUALITY ASSESSMENT")
print("="*80)

print("\n🔍 MISSING VALUES ANALYSIS")
print("-" * 40)

# Enhanced missing values analysis
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent,
    'Data Type': df.dtypes
}).sort_values('Missing Count', ascending=False)

# Only show columns with missing values
missing_cols = missing_df[missing_df['Missing Count'] > 0]
if len(missing_cols) > 0:
    print("Columns with missing values:")
    print(missing_cols)
else:
    print("✅ No missing values found in the dataset!")

print("\n🔄 DUPLICATE RECORDS ANALYSIS")
print("-" * 40)

# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows: {duplicate_count} ({duplicate_count/len(df)*100:.2f}%)")

if duplicate_count > 0:
    print("Sample duplicate rows:")
    print(df[df.duplicated()].head())

# Check for duplicate order numbers (business logic validation)
duplicate_orders = df['OrderNumber'].duplicated().sum()
print(f"Duplicate OrderNumbers: {duplicate_orders}")

print("\n🎯 DATA CONSISTENCY CHECKS")
print("-" * 40)

# Business logic validation
inconsistencies = []

# Check for negative values where they shouldn't exist
numeric_cols = ['Order Quantity', 'Unit Cost', 'Unit Price']
for col in numeric_cols:
    if col in df.columns:
        # Convert to numeric if necessary
        if df[col].dtype == 'object':
            # Remove currency symbols and commas, then convert
            temp_numeric = pd.to_numeric(df[col].replace(r'[\$,]', '', regex=True), errors='coerce')
        else:
            temp_numeric = df[col]
        negative_count = (temp_numeric < 0).sum()
        if negative_count > 0:
            inconsistencies.append(f"Negative {col}: {negative_count} records")

# Check discount range (should be 0-1)
if 'Discount Applied' in df.columns:
    invalid_discount = ((df['Discount Applied'] < 0) | (df['Discount Applied'] > 1)).sum()
    if invalid_discount > 0:
        inconsistencies.append(f"Invalid discount values: {invalid_discount} records")

# Check date logic (OrderDate <= ShipDate <= DeliveryDate)
if all(col in df.columns for col in ['OrderDate', 'ShipDate', 'DeliveryDate']):
    # Convert to datetime first for comparison
    df['OrderDate'] = pd.to_datetime(df['OrderDate'], format='mixed', dayfirst=True, errors='coerce')
    df['ShipDate'] = pd.to_datetime(df['ShipDate'], format='mixed', dayfirst=True, errors='coerce')
    df['DeliveryDate'] = pd.to_datetime(df['DeliveryDate'], format='mixed', dayfirst=True, errors='coerce')
    
    date_logic_issues = ((df['OrderDate'] > df['ShipDate']) | 
                        (df['ShipDate'] > df['DeliveryDate'])).sum()
    if date_logic_issues > 0:
        inconsistencies.append(f"Date logic violations: {date_logic_issues} records")

if inconsistencies:
    print("⚠️  Data inconsistencies found:")
    for issue in inconsistencies:
        print(f"   • {issue}")
else:
    print("✅ No major data inconsistencies found!")

In [ ]:
''' SECTION 3: STATISTICAL PROFILING '''
print("\n" + "="*80)
print("3. STATISTICAL PROFILING & DISTRIBUTION ANALYSIS")
print("="*80)

print("\n📊 SALES CHANNEL DISTRIBUTION")
print("-" * 40)

channel_stats = df.groupby('Sales Channel').agg({
    'OrderNumber': 'count',
    'Order Quantity': 'sum'
}).reset_index()

channel_stats['Percentage'] = (channel_stats['OrderNumber'] / len(df) * 100).round(2)
channel_stats = channel_stats.sort_values('OrderNumber', ascending=False)
print(channel_stats)

print("\n💰 FINANCIAL METRICS DISTRIBUTION")
print("-" * 40)

# Convert currency columns to numeric first
currency_columns = ['Unit Cost', 'Unit Price']
for col in currency_columns:
    if col in df.columns:
        df[col] = df[col].replace(r'[\$,]', '', regex=True).astype(float)

# Calculate financial metrics for profiling
if all(col in df.columns for col in ['Unit Price', 'Order Quantity', 'Discount Applied']):
    df['Revenue'] = df['Unit Price'] * df['Order Quantity'] * (1 - df['Discount Applied'])
    
    financial_stats = {
        'Total Revenue': df['Revenue'].sum(),
        'Average Order Value': df['Revenue'].mean(),
        'Median Order Value': df['Revenue'].median(),
        'Revenue Std Dev': df['Revenue'].std(),
        'Min Order Value': df['Revenue'].min(),
        'Max Order Value': df['Revenue'].max()
    }
    
    print("Financial Summary:")
    for metric, value in financial_stats.items():
        print(f"   • {metric}: ${value:,.2f}")

print("\n📅 DATE RANGE ANALYSIS")
print("-" * 40)

date_cols = ['OrderDate', 'ShipDate', 'DeliveryDate']
for col in date_cols:
    if col in df.columns:
        print(f"{col}:")
        print(f"   • Range: {df[col].min()} to {df[col].max()}")
        print(f"   • Span: {(df[col].max() - df[col].min()).days} days")

In [ ]:
''' SECTION 4: OUTLIER DETECTION '''
print("\n" + "="*80)
print("4. OUTLIER DETECTION & ANALYSIS")
print("="*80)

print("\n🔍 STATISTICAL OUTLIER DETECTION")
print("-" * 40)

def detect_outliers_iqr(series, column_name):
    """Detect outliers using IQR method"""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    
    return {
        'count': len(outliers),
        'percentage': len(outliers) / len(series) * 100,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }

# Check for outliers in key numeric columns
numeric_columns = ['Order Quantity', 'Unit Price', 'Unit Cost']
if 'Revenue' in df.columns:
    numeric_columns.append('Revenue')

outlier_summary = {}
for col in numeric_columns:
    if col in df.columns:
        outlier_info = detect_outliers_iqr(df[col], col)
        outlier_summary[col] = outlier_info
        print(f"{col}:")
        print(f"   • Outliers: {outlier_info['count']} ({outlier_info['percentage']:.2f}%)")
        print(f"   • Valid range: ${outlier_info['lower_bound']:.2f} to ${outlier_info['upper_bound']:.2f}")

In [ ]:
''' SECTION 5: ENHANCED FEATURE ENGINEERING '''
print("\n" + "="*80)
print("5. ENHANCED FEATURE ENGINEERING")
print("="*80)

def create_enhanced_features(df):
    """
    Create comprehensive feature set for advanced analytics
    """
    df_enhanced = df.copy()
    
    print("🔧 Creating financial metrics...")
    
    # Core financial calculations
    df_enhanced['Total Sales'] = df_enhanced['Unit Price'] * df_enhanced['Order Quantity']
    df_enhanced['Total Revenue'] = df_enhanced['Order Quantity'] * df_enhanced['Unit Price'] * (1 - df_enhanced['Discount Applied'])
    df_enhanced['Total Cost'] = df_enhanced['Unit Cost'] * df_enhanced['Order Quantity']
    df_enhanced['Total Profit'] = df_enhanced['Total Revenue'] - df_enhanced['Total Cost']
    df_enhanced['Profit Margin'] = np.where(df_enhanced['Total Revenue'] > 0, 
                                          df_enhanced['Total Profit'] / df_enhanced['Total Revenue'] * 100, 0)
    df_enhanced['Discount Amount'] = df_enhanced['Unit Price'] * df_enhanced['Discount Applied'] * df_enhanced['Order Quantity']
    
    print("📅 Creating date-based features...")
    
    # Enhanced date features
    for date_col, prefix in [('OrderDate', 'Order'), ('ShipDate', 'Ship'), ('DeliveryDate', 'Delivery')]:
        if date_col in df_enhanced.columns:
            df_enhanced[f'{prefix} Year'] = df_enhanced[date_col].dt.year
            df_enhanced[f'{prefix} Month'] = df_enhanced[date_col].dt.month
            df_enhanced[f'{prefix} Day'] = df_enhanced[date_col].dt.day
            df_enhanced[f'{prefix} Quarter'] = df_enhanced[date_col].dt.quarter
            df_enhanced[f'{prefix} DayOfWeek'] = df_enhanced[date_col].dt.dayofweek
            df_enhanced[f'{prefix} WeekOfYear'] = df_enhanced[date_col].dt.isocalendar().week
    
    print("⏱️ Creating operational metrics...")
    
    # Operational efficiency metrics
    if all(col in df_enhanced.columns for col in ['OrderDate', 'ShipDate', 'DeliveryDate']):
        df_enhanced['Processing_Days'] = (df_enhanced['ShipDate'] - df_enhanced['OrderDate']).dt.days
        df_enhanced['Shipping_Days'] = (df_enhanced['DeliveryDate'] - df_enhanced['ShipDate']).dt.days
        df_enhanced['Total_Lead_Time'] = (df_enhanced['DeliveryDate'] - df_enhanced['OrderDate']).dt.days
        
        # Categorize delivery performance
        df_enhanced['Delivery_Category'] = pd.cut(df_enhanced['Total_Lead_Time'], 
                                                bins=[0, 3, 7, 14, float('inf')],
                                                labels=['Express', 'Standard', 'Slow', 'Very Slow'])
    
    print("🎯 Creating business intelligence features...")
    
    # Customer value segmentation preparation
    df_enhanced['Order_Value_Category'] = pd.cut(df_enhanced['Total Revenue'], 
                                               bins=5, labels=['Low', 'Below Avg', 'Average', 'Above Avg', 'High'])
    
    # Product performance indicators
    df_enhanced['High_Margin_Product'] = df_enhanced['Profit Margin'] > df_enhanced['Profit Margin'].median()
    df_enhanced['Discounted_Order'] = df_enhanced['Discount Applied'] > 0
    df_enhanced['Large_Order'] = df_enhanced['Order Quantity'] > df_enhanced['Order Quantity'].quantile(0.75)
    
    # Seasonal indicators
    df_enhanced['Is_Holiday_Season'] = df_enhanced['Order Month'].isin([11, 12, 1])  # Nov, Dec, Jan
    df_enhanced['Is_Summer'] = df_enhanced['Order Month'].isin([6, 7, 8])  # Jun, Jul, Aug
    
    print("✅ Feature engineering completed!")
    print(f"   • Features added: {len(df_enhanced.columns) - len(df.columns)}")
    print(f"   • Total features: {len(df_enhanced.columns)}")
    
    return df_enhanced

# Apply enhanced feature engineering
df_enhanced = create_enhanced_features(df)

In [ ]:
''' SECTION 6: DATA VALIDATION & QUALITY REPORT '''
print("\n" + "="*80)
print("6. DATA VALIDATION & QUALITY REPORT")
print("="*80)

def generate_quality_report(df):
    """Generate comprehensive data quality report"""
    
    quality_metrics = {
        'Total Records': len(df),
        'Total Features': len(df.columns),
        'Memory Usage (MB)': df.memory_usage(deep=True).sum() / 1024**2,
        'Duplicate Records': df.duplicated().sum(),
        'Missing Values': df.isnull().sum().sum(),
        'Data Types': df.dtypes.value_counts().to_dict()
    }
    
    print("📋 DATA QUALITY SUMMARY")
    print("-" * 30)
    for metric, value in quality_metrics.items():
        if metric != 'Data Types':
            print(f"   • {metric}: {value}")
    
    print("\n📊 DATA TYPE DISTRIBUTION")
    print("-" * 30)
    for dtype, count in quality_metrics['Data Types'].items():
        print(f"   • {dtype}: {count} columns")
    
    # Feature categories
    numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = df.select_dtypes(include=['object']).columns.tolist()
    datetime_features = df.select_dtypes(include=['datetime64']).columns.tolist()
    
    print(f"\n🔢 FEATURE CATEGORIES")
    print("-" * 30)
    print(f"   • Numeric features: {len(numeric_features)}")
    print(f"   • Categorical features: {len(categorical_features)}")
    print(f"   • DateTime features: {len(datetime_features)}")
    
    return quality_metrics

quality_report = generate_quality_report(df_enhanced)

In [ ]:
''' SECTION 7: SAVE PROCESSED DATA '''
print("\n" + "="*80)
print("7. SAVING PROCESSED DATA")
print("="*80)

# Save the fully cleaned and enhanced dataset
output_path = '../data/processed/fully_cleaned_us_regional_sales_data.csv'
df_enhanced.to_csv(output_path, index=False)

print(f"✅ Enhanced dataset saved successfully!")
print(f"📍 Location: {output_path}")
print(f"📊 Final shape: {df_enhanced.shape}")

# Save data quality report
report_path = '../reports/data_quality_report.txt'
with open(report_path, 'w') as f:
    f.write("DATA QUALITY REPORT\n")
    f.write("="*50 + "\n\n")
    f.write(f"Generated: {datetime.now()}\n\n")
    
    f.write("DATASET OVERVIEW:\n")
    f.write(f"- Records: {len(df_enhanced):,}\n")
    f.write(f"- Features: {len(df_enhanced.columns)}\n")
    f.write(f"- Memory: {df_enhanced.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n\n")
    
    f.write("DATA QUALITY METRICS:\n")
    for metric, value in quality_report.items():
        if metric != 'Data Types':
            f.write(f"- {metric}: {value}\n")

print(f"📋 Data quality report saved: {report_path}")

print("\n" + "="*80)
print("ENHANCED EDA & DATA CLEANING COMPLETE")
print("="*80)

print(f"""
🎯 SUMMARY:
   • Processed {len(df_enhanced):,} records
   • Created {len(df_enhanced.columns) - len(df.columns)} additional features
   • Data quality validated and documented
   • Ready for advanced analytics!

📁 OUTPUT FILES:
   • Enhanced dataset: fully_cleaned_us_regional_sales_data.csv
   • Quality report: data_quality_report.txt
""")